In [1]:
import pandas as pd
from Bio import SeqIO
import gzip
import matplotlib.pyplot as plt
import numpy as np
import math
from functools import reduce
from IPython.display import Image

# Selección de regiones del genoma de N. benthamiana en base a su perfil transcriptómico

### Step 1:
Importamos un archivo con los identificadores de los genes de Niben261 con valores de expresión (gen correspondiente en Niben101), la posición inicial y final en el cromosoma, cromosoma y categoría de expresión a la que pertenecce y un archivo con todos los genes ano.

In [2]:
data = pd.read_csv("./raw_data/data_quintile_round_inNiben261.txt", sep = "\t", header = 0)
Niben261 = pd.read_csv("./raw_data/Niben261_genes.txt", sep = "\t", header = 0, index_col = 0)
Niben261chr = Niben261[Niben261.index.str.contains("Chr")]
# Complete data with Niben261 genes.
quintile_genes = data.index.tolist() # genes clasificados, con correspondencia en Niben101.
add_genes = list(set(Niben261chr.index.tolist()).difference(quintile_genes))
Niben261add = Niben261chr[Niben261chr.index.isin(add_genes)]
Niben261add["Chr"] = Niben261add.Chr.str.replace("Niben261Chr", "Chr")
data_complete = data.append(Niben261add)
data_complete

C:\Users\milil\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Chr,Start,End,quintile
Niben261Chr01g0002008,Chr01,266317,268851,1.0
Niben261Chr01g0004001,Chr01,458230,462273,3.0
Niben261Chr01g0004003,Chr01,467574,480423,2.0
Niben261Chr01g0004011,Chr01,465674,466936,1.0
Niben261Chr01g0004012,Chr01,432045,433043,1.0
...,...,...,...,...
Niben261Chr19g1484002,Chr19,148454367,148463060,NaN
Niben261Chr19g1485005,Chr19,148480135,148499120,NaN
Niben261Chr19g1485014,Chr19,148530961,148538381,NaN
Niben261Chr19g1486029,Chr19,148623104,148631519,NaN


### Step 2:
Longitud para cada cromosoma.

In [3]:
chrs = sorted(list(set(data_complete["Chr"]))) # List of chromosomes.

El dict se obtiene ejecutando esto, pero tarda mucho:

Niben261fasta = "C:/Users/milil/Downloads/Nextcloud/Documents/Miriam/Javi/Expression/Database/Genome/v261/Niben261_genome.fasta.gz"
chr_len = {} # Dict with chromosomes length.
with gzip.open(Niben261fasta, "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        chr_len[str(record.id)] = len(str(record.seq))

In [4]:
chr_len = {'Niben261Chr01': 186295347,
 'Niben261Chr02': 151625522,
 'Niben261Chr03': 194605305,
 'Niben261Chr04': 189815799,
 'Niben261Chr05': 138451958,
 'Niben261Chr06': 148966868,
 'Niben261Chr07': 152648067,
 'Niben261Chr08': 137820123,
 'Niben261Chr09': 137640743,
 'Niben261Chr10': 138251820,
 'Niben261Chr11': 143669543,
 'Niben261Chr12': 158218613,
 'Niben261Chr13': 153652170,
 'Niben261Chr14': 153778118,
 'Niben261Chr15': 153112746,
 'Niben261Chr16': 148817306,
 'Niben261Chr17': 155116228,
 'Niben261Chr18': 148425545,
 'Niben261Chr19': 148948562}

### Step 3:
**Estudio de los cromosomas para selección de ventana**

Conclusiones sobre las interspace zones:
* La ventana para estudiar la idoneidad de una región en base a los datos de expresión tendrá que abarcar por lo menos varios de ellos.
* La media de las interspace zones más grandes es de 787,578.94.
* Si tenemos una media de 44,701.88 bp en los cromosomas, la ventana tendrá que ser de varios cientos de miles de bp.

**Selección de parámetros**
* Ventana de 1000K.
* Coef [4, 2 , 1, -1, -2, -4].
* Eliminación de zonas sin genes.
* Al menos una región intergénica de 10K: lo cumplen todas.

*Los detalles están en score.ipynb*

### Step 4:
Cálculo de la idoneidad en el genoma.

In [5]:
def split(length, window):
    intervals = {}
    index = 1
    interval = []
    number_intervals = math.ceil(length/window)
    for i in range(length):
        interval.append(i + 1)
        if index == number_intervals:
            intervals[index] = [intervals[index - 1][1] + 1, length]
            break
        if len(interval) == window:
            intervals[index] = [min(interval), max(interval)]
            index += 1
            interval = []
    return(intervals)

In [ ]:
coef = {"x1": 4,
        "x2": 2,
        "x3": 1,
        "x4": -1,
        "x5": -2,
        "x6": -4}

window = 1000000

class Chr():
    def __init__(self, chr, length, window, data_complete, coef):
        self.chr = chr
        self.length = length
        self.window = window
        self.data_complete = data_complete
        self.coef = coef
        self.intervals_chr = split(length, window)
        self.idoneidad = self.idon()
        
    def idon(self):
        datachr = self.data_complete[self.data_complete["Chr"] == self.chr].sort_values('Start')
        
        self.idoneidad = {}
        for interval, values in self.intervals_chr.items():
            chr_values = datachr[(values[0] <= datachr["Start"]) & (datachr["Start"] < values[1])]
            n_genes = len(chr_values[chr_values['quintile'].notna()])
            if n_genes != 0:
                z = self.coef["x1"] * len(chr_values[chr_values["quintile"] == 5]) + self.coef["x2"] * len(chr_values[chr_values["quintile"] == 4]) + self.coef["x3"] * len(chr_values[chr_values["quintile"] == 3]) + self.coef["x4"] * len(chr_values[chr_values["quintile"] == 2]) + self.coef["x5"] * len(chr_values[chr_values["quintile"] == 1]) + self.coef["x6"] * len(chr_values[chr_values["quintile"] == 0])
                self.idoneidad[str(values[0]) + "-" + str(values[1])] = z
        return(self.idoneidad)

In [ ]:
idoneidad_dict = {}
for chr in chrs:
    print(chr)
    idoneidad_dict[chr] = Chr(chr, chr_len["Niben261" + chr], window, data_complete, coef)

In [ ]:
data_temp_list = []
for k, v in idoneidad_dict.items():
    #print(v.idoneidad)
    data_temp = pd.DataFrame(v.idoneidad.items(), columns=['Interval', 'Suitability'])
    data_temp["Chr"] = k
    data_temp["Window"] = range(1, len(v.idoneidad.keys()) + 1, 1)
    data_temp_list.append(data_temp)

data_idoneidad = pd.concat(data_temp_list)
data_idoneidad[["Start", "End"]] = data_idoneidad["Interval"].str.split('-', 1, expand=True)

In [ ]:
#data_idoneidad.to_csv("./results/idoneidad_v4.txt", sep = "\t")

In [6]:
data_idoneidad = pd.read_csv("./results/idoneidad_v4.txt", sep = "\t") # Para no ejecutar la función que tarda mucho.

### Step 5:
Estudio de características de las regiones intergénicas y de la ventana para seleccionar mejor las ventanas, no solo en base a su idoneidad/Suitability.

Características:
* Longitud media de las IR de la ventana.
* Número de genes asignados a una categoría de expresión en la ventana.
* Número de genes totales en la ventana.
* % GC medio de las IR de la ventana.
* % N medio de las IR de la ventana.
* Número de IR de la ventana.

In [7]:
# Chrs seqs.

Niben261fasta = "C:/Users/milil/Downloads/Nextcloud/Documents/Miriam/Javi/Expression/Database/Genome/v261/Niben261_genome.fasta.gz"
chr_seq = {} # Dict with chromosomes seq.
with gzip.open(Niben261fasta, "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        chr_seq[str(record.id)] = str(record.seq)

In [69]:
class Chr_length():
    def __init__(self, chr, data_idoneidad, data_complete, chr_seq):
        self.chr = chr
        self.data_idoneidad = data_idoneidad
        self.data_complete = data_complete
        self.chr_seq = chr_seq
        self.length_intergenic, self.requisite, self.n_genes_quintile, self.n_genes_total, self.intergenic_regions, self.GC, self.N, self.n_intergenic_regions = self.idon_length()
        
    def idon_length(self):
        datachr = self.data_complete[self.data_complete["Chr"] == self.chr].sort_values('Start')
        data_idoneidadchr = self.data_idoneidad[self.data_idoneidad["Chr"] == self.chr]
        
        self.length_intergenic = {}
        self.requisite = {}
        self.n_genes_quintile = {}
        self.n_genes_total = {}
        self.intergenic_regions = {}
        self.GC = {}
        self.N = {}
        self.n_intergenic_regions = {}
        for index, row in data_idoneidadchr.iterrows():
            self.seq = {}
            start = int(row["Interval"].split("-")[0])
            end = int(row["Interval"].split("-")[1])
            list_intergenic_length = []
            list_intergenic_GC = []
            list_intergenic_N = []
            requisite = 0 # Si tienen un región intergénica de al menos 10K será igual a 1.

            chr_subset = datachr[(start <= datachr["Start"]) & (datachr["Start"] < end)].sort_values("Start")
            if len(chr_subset) == 0:
                #list_intergenic_length.append(end-start + 1) # Se suman los extremos.
                # Si no hay genes en la ventana no se considera.
                pass

            else:
                count = 0
                n_IR = 0
                for i, r in chr_subset.iterrows():
                    if count == 0:
                        if r["Start"] > start:
                            intergenic_length = r["Start"] - start
                            list_intergenic_length.append(intergenic_length)
                            intergenic_seq = self.chr_seq["Niben261" + self.chr][start-1:r["Start"]-1].upper()
                            n_IR += 1
                            self.seq["IR_" + str(n_IR) + ";" + str(start) + "-" + str(r["Start"]-1)] = intergenic_seq.upper()
                            GC_content = ((intergenic_seq.count('G')+intergenic_seq.count('C'))/len(intergenic_seq))*100
                            list_intergenic_GC.append(GC_content)
                            N_content = (intergenic_seq.count('N')/len(intergenic_seq))*100
                            list_intergenic_N.append(N_content)
                            #intergenic_seq_GC = intergenic_seq # [x:y] la y se cuenta y-1, así que se deja x-1 e y porque la seq empieza por 0 y no por 1.
                        end_new = r["End"]
                    else:
                        intergenic_length = r["Start"] - end_new - 1 # No coger los extremos. Sólo en el caso de las IR entre genes, en el caso de la IR del principio y del final no hace falta.
                        if intergenic_length > 0:
                            list_intergenic_length.append(intergenic_length)
                            intergenic_seq = self.chr_seq["Niben261" + self.chr][end_new:r["Start"]-1].upper()
                            GC_content = ((intergenic_seq.count('G')+intergenic_seq.count('C'))/len(intergenic_seq))*100
                            list_intergenic_GC.append(GC_content)
                            N_content = (intergenic_seq.count('N')/len(intergenic_seq))*100
                            list_intergenic_N.append(N_content)
                            #intergenic_seq_GC = intergenic_seq
                            n_IR += 1
                            self.seq["IR_" + str(n_IR) + ";" + str(end_new + 1) + "-" + str(r["Start"]-1)] = intergenic_seq
                        end_new = r["End"]
                    #list_intergenic_length.append(intergenic_length)
                    count += 1
                    #self.seq["IR_" + str(index)] = intergenic_seq
                    if count == len(chr_subset):
                        if r["End"] < end:
                            intergenic_length = end - r["End"] 
                            intergenic_seq = self.chr_seq["Niben261" + self.chr][r["End"]:end].upper()
                            GC_content = ((intergenic_seq.count('G')+intergenic_seq.count('C'))/len(intergenic_seq))*100
                            #intergenic_seq_GC = intergenic_seq
                            list_intergenic_GC.append(GC_content)
                            N_content = (intergenic_seq.count('N')/len(intergenic_seq))*100
                            list_intergenic_N.append(N_content)
                            list_intergenic_length.append(intergenic_length)
                            n_IR += 1
                            self.seq["IR_" + str(n_IR) + ";" + str(r["End"]+1) + "-" + str(end)] = intergenic_seq
                if any(i >= 10000 for i in list_intergenic_length): requisite = 1
                #self.length_intergenic[row["Interval"]] = sum(list_intergenic_length)
                self.length_intergenic[row["Interval"]] = np.mean(list_intergenic_length)
                self.requisite[row["Interval"]] = requisite
                self.n_genes_quintile[row["Interval"]] = len(chr_subset[chr_subset['quintile'].notna()]) # n of genes assigned to one expression category.
                self.n_genes_total[row["Interval"]] = len(chr_subset) # n of total genes.
                self.intergenic_regions[row["Interval"]] = self.seq
                self.GC[row["Interval"]] = np.mean(list_intergenic_GC)
                self.N[row["Interval"]] = np.mean(list_intergenic_N)
                self.n_intergenic_regions[row["Interval"]] = n_IR
        return(self.length_intergenic, self.requisite, self.n_genes_quintile, self.n_genes_total, self.intergenic_regions, self.GC, self.N, self.n_intergenic_regions)

In [70]:
idoneidad_intergenic = {}
for chr in chrs:
    print(chr)
    idoneidad_intergenic[chr] = Chr_length(chr, data_idoneidad, data_complete, chr_seq)

Chr01
Chr02
Chr03
Chr04
Chr05
Chr06
Chr07
Chr08
Chr09
Chr10
Chr11
Chr12
Chr13
Chr14
Chr15
Chr16
Chr17
Chr18
Chr19


In [71]:
# Table 1:

data_length_temp_list = []
for k, v in idoneidad_intergenic.items():
    #print(v.idoneidad)
    data_temp_intergenic = pd.DataFrame(v.length_intergenic.items(), columns=['Interval', 'Mean Length IR'])
    data_temp_requisite = pd.DataFrame(v.requisite.items(), columns=['Interval', 'requisite'])
    data_temp_n_genes_quintile = pd.DataFrame(v.n_genes_quintile.items(), columns=['Interval', 'N genes assigned to expression category'])
    data_temp_n_genes_total = pd.DataFrame(v.n_genes_total.items(), columns=['Interval', 'N genes total'])
    data_temp_GC = pd.DataFrame(v.GC.items(), columns=['Interval', 'Mean % GC'])
    data_temp_N = pd.DataFrame(v.N.items(), columns=['Interval', 'Mean % N'])
    data_temp_n_intergenic_regions = pd.DataFrame(v.n_intergenic_regions.items(), columns=['Interval', 'N IR'])
    data_temp_intergenic["Chr"] = k
    dfs = [data_temp_intergenic, data_temp_requisite, data_temp_n_genes_quintile, data_temp_n_genes_total, data_temp_GC, data_temp_N, data_temp_n_intergenic_regions]
    df_temp = reduce(lambda left,right: pd.merge(left,right,on='Interval'), dfs)
    data_length_temp_list.append(df_temp)

data_intergenic = pd.concat(data_length_temp_list)
data_idoneidad_intergenic = pd.merge(data_idoneidad, data_intergenic, on=['Interval', 'Chr'], how='outer')
data_idoneidad_intergenic = data_idoneidad_intergenic[data_idoneidad_intergenic.columns[~data_idoneidad_intergenic.columns.str.contains('Unnamed:')]]
data_idoneidad_intergenic

,Interval,Suitability,Chr,Window,Start,End,Mean Length IR,requisite,N genes assigned to expression category,N genes total,Mean % GC,Mean % N,N IR
0,1-1000000,-11,Chr01,1,1,1000000,30352.793103,1,15,29,33.534990,9.317688,29
1,1000001-2000000,-26,Chr01,2,1000001,2000000,24140.558824,1,28,35,32.043833,9.104434,34
2,2000001-3000000,-15,Chr01,3,2000001,3000000,37669.916667,1,9,23,34.765797,5.991842,24
3,3000001-4000000,-30,Chr01,4,3000001,4000000,25096.171429,1,25,34,34.479037,8.040428,35
4,4000001-5000000,-7,Chr01,5,4000001,5000000,73944.153846,1,9,13,33.628559,9.327155,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935,144000001-145000000,-29,Chr19,145,144000001,145000000,37572.166667,1,17,23,33.537813,10.177448,24
2936,145000001-146000000,-37,Chr19,146,145000001,146000000,17201.872340,1,33,46,34.315892,6.056474,47
2937,146000001-147000000,-14,Chr19,147,146000001,147000000,31505.962963,1,19,26,32.106336,12.715414,27
2938,147000001-148000000,-14,Chr19,148,147000001,148000000,41179.409091,1,17,22,32.165613,11.726650,22


In [73]:
#data_idoneidad_intergenic.to_csv("./results/findIR_complete_v2.txt", sep = "\t")

In [4]:
# File with all the data.
data_idoneidad_intergenic = pd.read_csv("./results/findIR_complete_v2.txt", sep = "\t")

In [5]:
data_idoneidad_intergenic.sort_values('Suitability', ascending = False)

,Unnamed: 0,Interval,Suitability,Chr,Window,Start,End,Mean Length IR,requisite,N genes assigned to expression category,N genes total,Mean % GC,Mean % N,N IR
1895,1895,17000001-18000000,9,Chr13,18,17000001,18000000,41276.666667,1,14,20,35.146000,5.736527,21
1584,1584,9000001-10000000,6,Chr11,10,9000001,10000000,73755.461538,1,6,13,36.228817,7.296826,13
1426,1426,127000001-128000000,6,Chr09,128,127000001,128000000,36060.360000,1,16,24,34.396043,5.753853,25
1993,1993,115000001-116000000,6,Chr13,116,115000001,116000000,36698.080000,1,12,24,33.734135,5.430588,25
2157,2157,125000001-126000000,6,Chr14,126,125000001,126000000,71668.230769,1,10,12,33.145199,9.121484,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1867,148000001-149000000,-64,Chr12,149,148000001,149000000,20683.590909,1,27,43,40.883291,2.197469,44
2496,2496,9000001-10000000,-66,Chr17,10,9000001,10000000,32603.750000,1,23,27,32.587732,10.553918,28
2027,2027,149000001-150000000,-66,Chr13,150,149000001,150000000,25910.166667,1,20,37,42.441372,0.115343,36
2026,2026,148000001-149000000,-96,Chr13,149,148000001,149000000,23360.475000,1,26,41,43.659821,0.005470,40


### Step 6:
Generar fasta files con las secuencias de las regiones intergénicas por cromosoma y ventana.

Ellos podrán calcular el % GC, la longitud, el % N y lo que quieran desde el fasta.

In [76]:
for k, v in idoneidad_intergenic.items():
    fastaFile = open("./IR_seqs/" + k + "_IR.fasta", "w+")
    for window, IRs in v.intergenic_regions.items():
        for IR_id, IR_seq in IRs.items():
            fastaFile.write(">" + k + ";" + window + ";" + IR_id + "\n")
            fastaFile.write(IR_seq.upper() + "\n")
    fastaFile.close()

### Comprobaciones: para borrar después.

In [77]:
# Selección de varias ventanas para hacer las comprobaciones: la mejor (mayor suitability), la peor, dos intermedias, una en el principio de un cromosoma, otra en el final de otro cromosoma.
comprobaciones = {
    "Chr13_17000001-18000000": ("Chr13", 17000001, 18000000), # Mejor ventana.
    "Chr13_150000001-151000000": ("Chr13", 150000001, 151000000), # Peor ventana.
    "Chr02_142000001-143000000": ("Chr02", 142000001, 143000000), # Ventana intermedia.
    "Chr10_99000001-100000000": ("Chr10", 99000001, 100000000), # Ventana intermedia.
    "Chr07_1-1000000": ("Chr07", 1, 1000000), # Primera ventana de un cromosoma.
    "Chr05_138000001-138451958": ("Chr05", 138000001, 138451958), # Suitability = 0. Última ventana de un cromosoma.
}

In [78]:
# En findIR_compelte.txt, estas ventanas tienen los siguientes valores:
list_temp = []
for k, v in comprobaciones.items():
    chr_id = v[0]
    start = v[1]
    end = v[2]
    chr_data_temp = data_idoneidad_intergenic[data_idoneidad_intergenic["Chr"] == chr_id].sort_values('Start')
    temp = chr_data_temp[(chr_data_temp["Start"] >= start) & (chr_data_temp["Start"] < end)]
    list_temp.append(temp)
data_filter = pd.concat(list_temp)
data_filter

,Unnamed: 0,Interval,Suitability,Chr,Window,Start,End,Mean Length IR,requisite,N genes assigned to expression category,N genes total,Mean % GC,Mean % N,N IR
1895,1895,17000001-18000000,9,Chr13,18,17000001,18000000,41276.666667,1,14,20,35.146000,5.736527,21
2028,2028,150000001-151000000,-133,Chr13,151,150000001,151000000,13156.323529,1,40,68,43.222241,0.251912,68
329,329,142000001-143000000,-21,Chr02,143,142000001,143000000,64738.800000,1,7,14,32.489838,5.477867,15
1536,1536,99000001-100000000,-4,Chr10,100,99000001,100000000,53170.388889,1,8,17,34.719107,3.299987,18
1009,1009,1-1000000,-46,Chr07,1,1,1000000,23214.324324,1,25,36,34.037078,8.076054,37
860,860,138000001-138451958,0,Chr05,139,138000001,138451958,86555.200000,1,4,4,29.966651,21.803010,5


In [79]:
# Comprobación del cálculo de suitability.

coef = {"x1": 4, "x2":2, "x3":1, "x4":-1, "x5":-2, "x6":-4}

for k, v in comprobaciones.items():
    chr_id = v[0]
    start = v[1]
    end = v[2]
    chr_data = data_complete[data_complete["Chr"] == chr_id].sort_values('Start')
    chr_values = chr_data[(chr_data["Start"] >= start) & (chr_data["Start"] < end)]
    z = coef["x1"] * len(chr_values[chr_values["quintile"] == 5]) + coef["x2"] * len(chr_values[chr_values["quintile"] == 4]) + coef["x3"] * len(chr_values[chr_values["quintile"] == 3]) + coef["x4"] * len(chr_values[chr_values["quintile"] == 2]) + coef["x5"] * len(chr_values[chr_values["quintile"] == 1]) + coef["x6"] * len(chr_values[chr_values["quintile"] == 0])
    print("La ventana " + k + " tiene una z de " + str(z))
                

La ventana Chr13_17000001-18000000 tiene una z de 9
La ventana Chr13_150000001-151000000 tiene una z de -133
La ventana Chr02_142000001-143000000 tiene una z de -21
La ventana Chr10_99000001-100000000 tiene una z de -4
La ventana Chr07_1-1000000 tiene una z de -46
La ventana Chr05_138000001-138451958 tiene una z de 0


**El cálculo de suitability es correcto para las zonas seleccionadas.**

In [80]:
# Comprobación de la longitud de ventana.

for k, v in comprobaciones.items():
    chr_id = v[0]
    start = v[1]
    end = v[2]
    len_window = end - start + 1 # Se cuentan los extremos.
    print("La ventana " + k + " tiene una longitud de " + str(len_window))

La ventana Chr13_17000001-18000000 tiene una longitud de 1000000
La ventana Chr13_150000001-151000000 tiene una longitud de 1000000
La ventana Chr02_142000001-143000000 tiene una longitud de 1000000
La ventana Chr10_99000001-100000000 tiene una longitud de 1000000
La ventana Chr07_1-1000000 tiene una longitud de 1000000
La ventana Chr05_138000001-138451958 tiene una longitud de 451958


**El cálculo de la longitud de ventana es correcto para las zonas seleccionadas.**

In [81]:
# Comprobación del número de genes asignados a una categoría de expresión.

for k, v in comprobaciones.items():
    chr_id = v[0]
    start = v[1]
    end = v[2]
    chr_data = data_complete[data_complete["Chr"] == chr_id].sort_values('Start')
    chr_values = chr_data[(chr_data["Start"] >= start) & (chr_data["Start"] < end)]
    n_genes = len(chr_values[chr_values["quintile"].notna()])
    print("La ventana " + k + " tiene " + str(n_genes) + " asignados a categoría de expresión.")

La ventana Chr13_17000001-18000000 tiene 14 asignados a categoría de expresión.
La ventana Chr13_150000001-151000000 tiene 40 asignados a categoría de expresión.
La ventana Chr02_142000001-143000000 tiene 7 asignados a categoría de expresión.
La ventana Chr10_99000001-100000000 tiene 8 asignados a categoría de expresión.
La ventana Chr07_1-1000000 tiene 25 asignados a categoría de expresión.
La ventana Chr05_138000001-138451958 tiene 4 asignados a categoría de expresión.


**El cálculo del número de genes asignados a una categoría de expresión es correcto para las zonas seleccionadas.**

In [36]:
# Comprobación del número de genes totales.

for k, v in comprobaciones.items():
    chr_id = v[0]
    start = v[1]
    end = v[2]
    chr_data = data_complete[data_complete["Chr"] == chr_id].sort_values('Start')
    chr_values = chr_data[(chr_data["Start"] >= start) & (chr_data["Start"] < end)]
    n_genes = len(chr_values)
    print("La ventana " + k + " tiene " + str(n_genes) + " totales.")

La ventana Chr13_17000001-18000000 tiene 20 totales.
La ventana Chr13_150000001-151000000 tiene 68 totales.
La ventana Chr02_142000001-143000000 tiene 14 totales.
La ventana Chr10_99000001-100000000 tiene 17 totales.
La ventana Chr07_1-1000000 tiene 36 totales.
La ventana Chr05_138000001-138451958 tiene 4 totales.


**El cálculo del número de genes totales es correcto para las zonas seleccionadas.**

In [82]:
# Comprobación de número de IR.

for k, v in comprobaciones.items():
    chr_id = v[0]
    start = v[1]
    end = v[2]
    chr_data = data_complete[data_complete["Chr"] == chr_id].sort_values('Start')
    chr_values = chr_data[(chr_data["Start"] >= start) & (chr_data["Start"] < end)]
    
    n_IR = 0
    count = 0
    for index, row in chr_values.iterrows():
        if count == 0:
            if row["Start"] <= start: pass
            else:
                n_IR += 1
            new_end = row["End"]
        else:
            if row["Start"] > new_end + 1:
                n_IR += 1
            new_end = row["End"]
        count += 1
        if count == len(chr_values):
            if row["End"] < end: n_IR += 1
    print("La ventana " + k + " tiene " + str(n_IR) + " IR.")

La ventana Chr13_17000001-18000000 tiene 21 IR.
La ventana Chr13_150000001-151000000 tiene 68 IR.
La ventana Chr02_142000001-143000000 tiene 15 IR.
La ventana Chr10_99000001-100000000 tiene 18 IR.
La ventana Chr07_1-1000000 tiene 37 IR.
La ventana Chr05_138000001-138451958 tiene 5 IR.


**El cálculo del número de regiones intergénicas es correcto para las ventanas seleccionadas.**

Comprobado en Excel y en los fasta también.

In [84]:
# Comprobación de la longitud media de IR.

for k, v in comprobaciones.items():
    chr_id = v[0]
    start = v[1]
    end = v[2]
    chr_data = data_complete[data_complete["Chr"] == chr_id].sort_values('Start')
    chr_values = chr_data[(chr_data["Start"] >= start) & (chr_data["Start"] < end)]
    
    length_list = []
    count = 0
    for index, row in chr_values.iterrows():
        if count == 0:
            if row["Start"] <= start: pass
            else:
                length = row["Start"] - start
                length_list.append(length)
            new_end = row["End"]
        else:
            if row["Start"] > new_end + 1:
                length = row["Start"] - new_end - 1
                length_list.append(length)
            new_end = row["End"]
        count += 1
        if count == len(chr_values):
            if row["End"] < end:
                length = end - row["End"]
                length_list.append(length)
    mean_length = np.mean(length_list)
    print("La ventana " + k + " tiene " + str(mean_length) + " longitud media.")

La ventana Chr13_17000001-18000000 tiene 41276.666666666664 longitud media.
La ventana Chr13_150000001-151000000 tiene 13156.323529411764 longitud media.
La ventana Chr02_142000001-143000000 tiene 64738.8 longitud media.
La ventana Chr10_99000001-100000000 tiene 53170.38888888889 longitud media.
La ventana Chr07_1-1000000 tiene 23214.324324324323 longitud media.
La ventana Chr05_138000001-138451958 tiene 86555.2 longitud media.


In [86]:
# Comprobación de la longitud media de IR 2.

for k, v in comprobaciones.items():
    chr_id = v[0]
    start = v[1]
    end = v[2]
    chr_data = data_complete[data_complete["Chr"] == chr_id].sort_values('Start')
    chr_values = chr_data[(chr_data["Start"] >= start) & (chr_data["Start"] < end)]
    seq = chr_seq["Niben261" + chr_id]
    
    length_list = []
    count = 0
    for index, row in chr_values.iterrows():
        if count == 0:
            if row["Start"] <= start: pass
            else:
                IR_seq = seq[start-1:row["Start"]-1]
                length = len(IR_seq)
                length_list.append(length)
            new_end = row["End"]
        else:
            if row["Start"] > new_end + 1:
                IR_seq = seq[new_end:row["Start"]-1]
                length = len(IR_seq)
                length_list.append(length)
            new_end = row["End"]
        count += 1
        if count == len(chr_values):
            if row["End"] < end:
                IR_seq = seq[row["End"]:end]
                length = len(IR_seq)
                length_list.append(length)
    mean_length = np.mean(length_list)
    print("La ventana " + k + " tiene " + str(mean_length) + " longitud media.")

La ventana Chr13_17000001-18000000 tiene 41276.666666666664 longitud media.
La ventana Chr13_150000001-151000000 tiene 13156.323529411764 longitud media.
La ventana Chr02_142000001-143000000 tiene 64738.8 longitud media.
La ventana Chr10_99000001-100000000 tiene 53170.38888888889 longitud media.
La ventana Chr07_1-1000000 tiene 23214.324324324323 longitud media.
La ventana Chr05_138000001-138451958 tiene 86555.2 longitud media.


**El cálculo de la longitud media de las IR es correcto para las ventanas seleccionadas.**

In [87]:
# Comprobación del requisito de las ventanas.

for k, v in comprobaciones.items():
    chr_id = v[0]
    start = v[1]
    end = v[2]
    chr_data = data_complete[data_complete["Chr"] == chr_id].sort_values('Start')
    chr_values = chr_data[(chr_data["Start"] >= start) & (chr_data["Start"] < end)]
    seq = chr_seq["Niben261" + chr_id]
    requisite = 0
    
    length_list = []
    count = 0
    for index, row in chr_values.iterrows():
        if count == 0:
            if row["Start"] <= start: pass
            else:
                IR_seq = seq[start-1:row["Start"]-1]
                length = len(IR_seq)
                length_list.append(length)
            new_end = row["End"]
        else:
            if row["Start"] > new_end + 1:
                IR_seq = seq[new_end:row["Start"]-1]
                length = len(IR_seq)
                length_list.append(length)
            new_end = row["End"]
        count += 1
        if count == len(chr_values):
            if row["End"] < end:
                IR_seq = seq[row["End"]:end]
                length = len(IR_seq)
                length_list.append(length)
    if any(item >= 10000 for item in length_list): requisite = 1
    print("La ventana " + k + " tiene " + str(requisite) + " requisite.")

La ventana Chr13_17000001-18000000 tiene 1 requisite.
La ventana Chr13_150000001-151000000 tiene 1 requisite.
La ventana Chr02_142000001-143000000 tiene 1 requisite.
La ventana Chr10_99000001-100000000 tiene 1 requisite.
La ventana Chr07_1-1000000 tiene 1 requisite.
La ventana Chr05_138000001-138451958 tiene 1 requisite.


**El cálculo del requisito es correcto para las ventanas seleccionadas.**

In [88]:
# Comprobación del contenido medio de % GC.

for k, v in comprobaciones.items():
    chr_id = v[0]
    start = v[1]
    end = v[2]
    chr_data = data_complete[data_complete["Chr"] == chr_id].sort_values('Start')
    chr_values = chr_data[(chr_data["Start"] >= start) & (chr_data["Start"] < end)]
    seq = chr_seq["Niben261" + chr_id]
    
    GC_list = []
    count = 0
    for index, row in chr_values.iterrows():
        if count == 0:
            if row["Start"] <= start: pass
            else:
                IR_seq = seq[start-1:row["Start"]-1].upper()
                GC_content = ((IR_seq.count('G')+IR_seq.count('C'))/len(IR_seq))*100
                GC_list.append(GC_content)
            new_end = row["End"]
        else:
            if row["Start"] > new_end + 1:
                IR_seq = seq[new_end:row["Start"]-1].upper()
                GC_content = ((IR_seq.count('G')+IR_seq.count('C'))/len(IR_seq))*100
                GC_list.append(GC_content)
            new_end = row["End"]
        count += 1
        if count == len(chr_values):
            if row["End"] < end:
                IR_seq = seq[row["End"]:end].upper()
                GC_content = ((IR_seq.count('G')+IR_seq.count('C'))/len(IR_seq))*100
                GC_list.append(GC_content)
    mean_GC = np.mean(GC_list)
    print("La ventana " + k + " tiene " + str(mean_GC) + " % GC.")

La ventana Chr13_17000001-18000000 tiene 35.14599982080838 % GC.
La ventana Chr13_150000001-151000000 tiene 43.222241072760454 % GC.
La ventana Chr02_142000001-143000000 tiene 32.48983816459409 % GC.
La ventana Chr10_99000001-100000000 tiene 34.71910695483638 % GC.
La ventana Chr07_1-1000000 tiene 34.03707807411949 % GC.
La ventana Chr05_138000001-138451958 tiene 29.96665056144073 % GC.


**El cálculo del % GC medio es correcto para las ventanas seleccionadas.**

In [91]:
# Comprobación del contenido medio de % N.

for k, v in comprobaciones.items():
    chr_id = v[0]
    start = v[1]
    end = v[2]
    chr_data = data_complete[data_complete["Chr"] == chr_id].sort_values('Start')
    chr_values = chr_data[(chr_data["Start"] >= start) & (chr_data["Start"] < end)]
    seq = chr_seq["Niben261" + chr_id]
    
    N_list = []
    count = 0
    for index, row in chr_values.iterrows():
        if count == 0:
            if row["Start"] <= start: pass
            else:
                IR_seq = seq[start-1:row["Start"]-1].upper()
                N_content = (IR_seq.count('N')/len(IR_seq))*100
                N_list.append(N_content)
            new_end = row["End"]
        else:
            if row["Start"] > new_end + 1:
                IR_seq = seq[new_end:row["Start"]-1].upper()
                N_content = (IR_seq.count('N')/len(IR_seq))*100
                N_list.append(N_content)
            new_end = row["End"]
        count += 1
        if count == len(chr_values):
            if row["End"] < end:
                IR_seq = seq[row["End"]:end].upper()
                N_content = (IR_seq.count('N')/len(IR_seq))*100
                N_list.append(N_content)
    mean_N = np.mean(N_list)
    print("La ventana " + k + " tiene " + str(mean_N) + " % N.")

La ventana Chr13_17000001-18000000 tiene 5.736526750546687 % N.
La ventana Chr13_150000001-151000000 tiene 0.2519120160289101 % N.
La ventana Chr02_142000001-143000000 tiene 5.477867335809547 % N.
La ventana Chr10_99000001-100000000 tiene 3.299987325502919 % N.
La ventana Chr07_1-1000000 tiene 8.0760538780196 % N.
La ventana Chr05_138000001-138451958 tiene 21.803010408307518 % N.


**El cálculo del % N medio es correcto para las ventanas seleccionadas.**

In [15]:
chr_id = "Chr05"
start = 138000001
end = 138451958

chr_data = data_complete[data_complete["Chr"] == chr_id].sort_values('Start')
chr_values = chr_data[(chr_data["Start"] >= start) & (chr_data["Start"] < end)]
chr_values

,Chr,Start,End,quintile
Niben261Chr05g1381005,Chr05,138148875,138153885,4.0
Niben261Chr05g1381006,Chr05,138154993,138161476,2.0
Niben261Chr05g1382004,Chr05,138194697,138200365,3.0
Niben261Chr05g1383003,Chr05,138338765,138340782,1.0


In [7]:
comprobaciones = {
    "Chr13_17000001-18000000": ("Chr13", 17000001, 18000000), # Mejor ventana.
    "Chr13_150000001-151000000": ("Chr13", 150000001, 151000000), # Peor ventana.
    "Chr02_142000001-143000000": ("Chr02", 142000001, 143000000), # Ventana intermedia.
    "Chr10_99000001-100000000": ("Chr10", 99000001, 100000000), # Ventana intermedia.
    "Chr07_1-1000000": ("Chr07", 1, 1000000), # Primera ventana de un cromosoma.
    "Chr05_138000001-138451958": ("Chr05", 138000001, 138451958), # Suitability = 0. Última ventana de un cromosoma.
}

In [8]:
# En findIR_compelte.txt, estas ventanas tienen los siguientes valores:
list_temp = []
for k, v in comprobaciones.items():
    chr_id = v[0]
    start = v[1]
    end = v[2]
    chr_data_temp = data_idoneidad_intergenic[data_idoneidad_intergenic["Chr"] == chr_id].sort_values('Start')
    temp = chr_data_temp[(chr_data_temp["Start"] >= start) & (chr_data_temp["Start"] < end)]
    list_temp.append(temp)
data_filter = pd.concat(list_temp)
data_filter

,Unnamed: 0,Interval,Suitability,Chr,Window,Start,End,Mean Length IR,requisite,N genes assigned to expression category,N genes total,Mean % GC,Mean % N,N IR
1895,1895,17000001-18000000,9,Chr13,18,17000001,18000000,41276.666667,1,14,20,35.146000,5.736527,21
2028,2028,150000001-151000000,-133,Chr13,151,150000001,151000000,13156.323529,1,40,68,43.222241,0.251912,68
329,329,142000001-143000000,-21,Chr02,143,142000001,143000000,64738.800000,1,7,14,32.489838,5.477867,15
1536,1536,99000001-100000000,-4,Chr10,100,99000001,100000000,53170.388889,1,8,17,34.719107,3.299987,18
1009,1009,1-1000000,-46,Chr07,1,1,1000000,23214.324324,1,25,36,34.037078,8.076054,37
860,860,138000001-138451958,0,Chr05,139,138000001,138451958,86555.200000,1,4,4,29.966651,21.803010,5
